<h1 align=center>Proyecto: Titanic - Machine Learning from Disaster</h1>
<hr>
<h2 align=center>Probamos modelos de ML</h2>

**Importamos librerias.**

In [4]:
import pandas as pd
import sklearn as sk
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler, LabelEncoder
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
import pickle


ModuleNotFoundError: No module named 'faiss'

<h3 align=center>1.-Realizamos la lectura de los datos y los guardamos en sus respectivos dataframes.</h3>

In [5]:
data_test = pd.read_csv("../datasets/test.csv")
data_train = pd.read_csv("../datasets/train.csv")

In [6]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [33]:
#data_train["Age"].fillna(data_train["Age"].median(),inplace=True)
#data_test["Age"].fillna(data_test["Age"].median(),inplace=True)

data_test["Fare"].fillna(data_test["Fare"].median(),inplace=True)
"""le = LabelEncoder()
le.fit(data_train['Cabin'])
data_train['Cabin']= le.transform(data_train['Cabin'])"""

"le = LabelEncoder()\nle.fit(data_train['Cabin'])\ndata_train['Cabin']= le.transform(data_train['Cabin'])"

In [34]:
data_train['Cabin'].unique()

array([nan, 'C85', 'C123', 'E46', 'G6', 'C103', 'D56', 'A6',
       'C23 C25 C27', 'B78', 'D33', 'B30', 'C52', 'B28', 'C83', 'F33',
       'F G73', 'E31', 'A5', 'D10 D12', 'D26', 'C110', 'B58 B60', 'E101',
       'F E69', 'D47', 'B86', 'F2', 'C2', 'E33', 'B19', 'A7', 'C49', 'F4',
       'A32', 'B4', 'B80', 'A31', 'D36', 'D15', 'C93', 'C78', 'D35',
       'C87', 'B77', 'E67', 'B94', 'C125', 'C99', 'C118', 'D7', 'A19',
       'B49', 'D', 'C22 C26', 'C106', 'C65', 'E36', 'C54',
       'B57 B59 B63 B66', 'C7', 'E34', 'C32', 'B18', 'C124', 'C91', 'E40',
       'T', 'C128', 'D37', 'B35', 'E50', 'C82', 'B96 B98', 'E10', 'E44',
       'A34', 'C104', 'C111', 'C92', 'E38', 'D21', 'E12', 'E63', 'A14',
       'B37', 'C30', 'D20', 'B79', 'E25', 'D46', 'B73', 'C95', 'B38',
       'B39', 'B22', 'C86', 'C70', 'A16', 'C101', 'C68', 'A10', 'E68',
       'B41', 'A20', 'D19', 'D50', 'D9', 'A23', 'B50', 'A26', 'D48',
       'E58', 'C126', 'B71', 'B51 B53 B55', 'D49', 'B5', 'B20', 'F G63',
       'C62 C64',

In [35]:

# Vemos la posilibilad de establecer la edad en rangos.
bins= [0,10,20,30,40,50,60,100]
names = ['0','1','2','3','4','5','6']
data_train['Age'] = pd.cut(data_train['Age'], bins, labels=names)
data_test['Age'] = pd.cut(data_test['Age'], bins, labels=names)
data_train = pd.concat([data_train ,pd.get_dummies(data_train['Age'])], axis=1 )
data_test = pd.concat([data_test ,pd.get_dummies(data_test['Age'])], axis=1 )

<h3 align=center>2.-Realizamos pipelines.</h3>

Establecemos el modelo a utilizar. Para cambiar el modelo solo basta con cambiarlo en la siguiente linea de código sin que sea necesario alterar pipelines.

In [84]:
model = DecisionTreeClassifier(criterion= 'gini', max_depth= 8,min_samples_leaf= 1, min_samples_split= 1)
#model = KNeighborsClassifier()
#model = BaggingClassifier()
#model = RandomForestClassifier()
#model = SVC()

Separamos el set de entrenamiento y testeo.

In [37]:
x= data_train[['Pclass', 'Sex', 'Fare','Embarked']]
y= data_train['Survived']


In [38]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=60, stratify=y)

Establecemos por medio de variables que procesos de transformacion se le aplicaran a las diversas columnas. Para cambiar, solo basta con modificar el contenido de las variables sin que sea necesario alterar pipelines.

In [39]:
nominal =['Sex','Embarked' ]
ordinal =['Pclass' ]
numerical = ['Fare']

Definimos pipeline relativos a la tranformación  de los datos.

In [85]:
#Pipeline datos numéricos.
numerical_pipeline = Pipeline([('escaler', StandardScaler())])
#Pipeline datos ordinales. 
ordinal_pipeline = Pipeline([("encoder", OrdinalEncoder())])
#Pipeline datos nominales.
nominal_pipeline = Pipeline([("encoder2", OneHotEncoder())])

#Unimos los dos procesos en un mismo Pipeline.
preprocessin_pipeline = ColumnTransformer([("ordinal_preprocesor", ordinal_pipeline, ordinal),
                                            ("nominal_preprocessor", nominal_pipeline, nominal),
                                            ("numerical_preprocessor",numerical_pipeline, numerical) ])

Realizamos el pipeline completo.

In [86]:
complete_pipeline= Pipeline ([("preprocessor", preprocessin_pipeline), ("estimator", model)])

<h3 align=center>3.-Probamos los modelos.</h3>

**A.-Utilizamos GridSearch para orientarnos con la elección de hiperparametros.**

In [42]:
# Parametros DecisionTreeClassifier

params_tree = {
             'estimator__criterion' : ['gini', 'entropy'],
             'estimator__min_samples_split':range(1,10),
             'estimator__min_samples_leaf':range(1,10),
             'estimator__max_depth':range(1,20) 
}


In [43]:
model = GridSearchCV(complete_pipeline, param_grid=params_tree, cv=5)#, scoring = ['accuracy', 'recall'], refit = 'accuracy' )
model.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('ordinal_preprocesor',
                                                                         Pipeline(steps=[('encoder',
                                                                                          OrdinalEncoder())]),
                                                                         ['Pclass']),
                                                                        ('nominal_preprocessor',
                                                                         Pipeline(steps=[('encoder2',
                                                                                          OneHotEncoder())]),
                                                                         ['Sex',
                                                                          'Embarked']),
                                                                        ('numerical_preprocessor',
                                                                         Pipeline(steps=[('escaler',
                                                                                          StandardScaler())]),
                                                                         ['Fare'])])),
                                       ('estimator',
                                        DecisionTreeClassifier())]),
             param_grid={'estimator__criterion': ['gini', 'entropy'],
                         'estimator__max_depth': range(1, 20),
                         'estimator__min_samples_leaf': range(1, 10),
                         'estimator__min_samples_split': range(1, 10)})

In [44]:
print("Mejores hiperparámetros: "+str(model.best_params_))
print("Mejor Score: "+str(model.best_score_)+'\n')

scores = pd.DataFrame(model.cv_results_)
scores
#'estimator__criterion': 'gini', 'estimator__max_depth': 8, 'estimator__min_samples_leaf': 1, 'estimator__min_samples_split': 1

Mejores hiperparámetros: {'estimator__criterion': 'gini', 'estimator__max_depth': 8, 'estimator__min_samples_leaf': 1, 'estimator__min_samples_split': 1}
Mejor Score: 0.8089924160346695



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_estimator__criterion,param_estimator__max_depth,param_estimator__min_samples_leaf,param_estimator__min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.016581,0.002761,0.009289,0.003776,gini,1,1,1,"{'estimator__criterion': 'gini', 'estimator__m...",0.769231,0.762238,0.746479,0.823944,0.809859,0.782350,0.029496,2350
1,0.019092,0.002964,0.009950,0.002321,gini,1,1,2,"{'estimator__criterion': 'gini', 'estimator__m...",0.769231,0.762238,0.746479,0.823944,0.809859,0.782350,0.029496,2350
2,0.019314,0.002184,0.007522,0.003867,gini,1,1,3,"{'estimator__criterion': 'gini', 'estimator__m...",0.769231,0.762238,0.746479,0.823944,0.809859,0.782350,0.029496,2350
3,0.013899,0.002684,0.006318,0.003394,gini,1,1,4,"{'estimator__criterion': 'gini', 'estimator__m...",0.769231,0.762238,0.746479,0.823944,0.809859,0.782350,0.029496,2350
4,0.012499,0.003880,0.007296,0.004665,gini,1,1,5,"{'estimator__criterion': 'gini', 'estimator__m...",0.769231,0.762238,0.746479,0.823944,0.809859,0.782350,0.029496,2350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3073,0.010062,0.004114,0.003970,0.004456,entropy,19,9,5,"{'estimator__criterion': 'entropy', 'estimator...",0.769231,0.811189,0.788732,0.830986,0.795775,0.799183,0.020850,448
3074,0.009305,0.002428,0.001854,0.001914,entropy,19,9,6,"{'estimator__criterion': 'entropy', 'estimator...",0.769231,0.811189,0.788732,0.830986,0.795775,0.799183,0.020850,448
3075,0.006523,0.002330,0.004834,0.005535,entropy,19,9,7,"{'estimator__criterion': 'entropy', 'estimator...",0.769231,0.811189,0.788732,0.830986,0.795775,0.799183,0.020850,448
3076,0.013360,0.002861,0.001699,0.001771,entropy,19,9,8,"{'estimator__criterion': 'entropy', 'estimator...",0.769231,0.811189,0.788732,0.830986,0.795775,0.799183,0.020850,448


In [70]:
prediction = model.predict(X_test)
report = classification_report(y_test,prediction)
print("Reporte de Clasificación:")
print(report)

NotFittedError: This DecisionTreeClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

**B.-Probamos modelos teniendo en cuenta lo sugerido por Gridsearch.**

In [87]:
model =complete_pipeline.fit(X_train,y_train)

Evaluamos el desempeño del modelo.

In [88]:
prediction = complete_pipeline.predict(X_test)


In [89]:
cm = confusion_matrix(y_test,prediction)
print("Matriz de confusión:")
print(cm)

Matriz de confusión:
[[100  10]
 [ 17  52]]


In [90]:

report = classification_report(y_test,prediction)
print("Reporte de Clasificación:")
print(report)

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.85      0.91      0.88       110
           1       0.84      0.75      0.79        69

    accuracy                           0.85       179
   macro avg       0.85      0.83      0.84       179
weighted avg       0.85      0.85      0.85       179



**C.-Elegido el modelo procedemos a entrenarlo con el dataset completo de Train para enviar la predicción sobre los datos de test a Kaggle en formato .csv.**

In [91]:
model =complete_pipeline.fit(x,y)

In [92]:
prediction = pd.DataFrame()


In [93]:
x_test_completo = data_test[['Pclass', 'Sex', 'Age','Fare','Embarked', 'Parch']]
prediction['PassengerID'] = data_test['PassengerId']
prediction['Survived'] = complete_pipeline.predict(x_test_completo)

In [94]:
prediction.set_index('PassengerID')

,Survived
PassengerID,
892,0
893,1
894,0
895,0
896,1
...,...
1305,0
1306,1
1307,0


In [96]:
prediction.to_csv(path_or_buf='../Tree1.csv' , index=False)

In [55]:
"""
Para guardar modelos con pickle
pickle.dump(model, open('model.pkl', 'wb'))
pickled_model = pickle.load(open('model.pkl', 'rb'))
pickled_model.predict(X_test)
"""

"\nPara guardar modelos con pickle\npickle.dump(model, open('model.pkl', 'wb'))\npickled_model = pickle.load(open('model.pkl', 'rb'))\npickled_model.predict(X_test)\n"